# Tutorial: Multiple Linear Regression with Vehicle Data

## What you’ll build
A simple end-to-end regression workflow that:
1. Imports a small dataset from an online source
2. Makes it *intentionally messy* (like real-world data)
3. Cleans it (types, missing values, duplicates, basic outliers)
4. Encodes a categorical variable using `pd.get_dummies`
5. Scales features (standardization)
6. Trains/tests a **Multiple Linear Regression** model
7. Evaluates performance with MAE, RMSE, and R²

## Dataset
We’ll use the classic Auto MPG dataset.  
**Target:** `mpg` (miles per gallon)  
**Predictors (exactly 4):**
- Numeric: `horsepower`, `weight`, `model_year`
- Categorical: `origin`

> Note: `seaborn.load_dataset()` typically downloads data the first time, so you may need internet access for the import step.


In [48]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Newer scikit-learn versions provide RMSE directly:
try:
    from sklearn.metrics import root_mean_squared_error
    HAS_RMSE = True
except Exception:
    HAS_RMSE = False


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


## Step 1 — Import data

Why sample the dataset?
- Keeps the tutorial fast and readable
- Mimics “limited data availability” situations
- Makes debugging easier


In [50]:
df_raw = sns.load_dataset("mpg")  # loads from an online source behind the scenes

# Keep it small
df = df_raw.sample(160, random_state=7).copy()

# Keep only target + 4 predictors
df = df[["mpg", "horsepower", "weight", "model_year", "origin"]].copy()

# Add duplicates
df = pd.concat([df, df.sample(6, random_state=1)], ignore_index=True)

# Inject messiness: non-numeric horsepower + trailing spaces in weight + missing origin
idx = df.sample(12, random_state=2).index
df.loc[idx[:4], "horsepower"] = "unknown"
df.loc[idx[4:8], "horsepower"] = np.nan
df.loc[idx[8:10], "weight"] = df.loc[idx[8:10], "weight"].astype(str) + " "
df.loc[idx[10:], "origin"] = None

df.head(10)


/var/folders/1v/q657715s42sdnd2rpmmct5_r0000gn/T/ipykernel_82972/1429728668.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[idx[:4], "horsepower"] = "unknown"
/var/folders/1v/q657715s42sdnd2rpmmct5_r0000gn/T/ipykernel_82972/1429728668.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['4129 ' '2600 ']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[idx[8:10], "weight"] = df.loc[idx[8:10], "weight"].astype(str) + " "


,mpg,horsepower,weight,model_year,origin
0,26.0,75.0,2246,74,europe
1,22.0,112.0,2835,82,usa
2,34.7,63.0,2215,81,None
3,30.0,NaN,2065,71,europe
4,29.8,62.0,1845,80,europe
5,31.0,68.0,1970,82,japan
6,26.0,NaN,2391,74,japan
7,31.5,68.0,2045,77,japan
8,30.0,67.0,3250,80,europe
9,32.0,84.0,2295,82,usa


## Step 2 — Make the data messy on purpose

Real datasets often have:
- Duplicates
- Missing values
- Wrong types (numbers stored as strings)
- “Unknown” markers instead of nulls
- Extra whitespace

We’ll inject a few of these issues so the cleaning steps are meaningful.


In [52]:
# Add duplicates
df = pd.concat([df, df.sample(6, random_state=1)], ignore_index=True)

# Inject messiness:
# - horsepower: non-numeric strings and missing values
# - weight: trailing spaces (stringy numeric)
# - origin: missing category
idx = df.sample(12, random_state=2).index
df.loc[idx[:4], "horsepower"] = "unknown"
df.loc[idx[4:8], "horsepower"] = np.nan
df.loc[idx[8:10], "weight"] = df.loc[idx[8:10], "weight"].astype(str) + " "
df.loc[idx[10:], "origin"] = None

df.head(10)


,mpg,horsepower,weight,model_year,origin
0,26.0,75.0,2246,74,europe
1,22.0,112.0,2835,82,usa
2,34.7,63.0,2215,81,None
3,30.0,NaN,2065,71,None
4,29.8,62.0,1845,80,europe
5,31.0,68.0,1970,82,japan
6,26.0,NaN,2391,74,japan
7,31.5,68.0,2045,77,japan
8,30.0,67.0,3250,80,europe
9,32.0,84.0,2295,82,usa


## Step 3 — Analysis (quick, practical checks)

Before cleaning, always inspect:
- Shape (rows/columns)
- Data types (are numbers stored as strings?)
- Missing values (how many and where?)
- Duplicates
- Category distribution (rare categories, missing)
- Sanity-check values (a few raw examples)


In [55]:
print("Shape:", df.shape)

Shape: (172, 5)


In [57]:
print("\nDtypes:\n", df.dtypes)


Dtypes:
 mpg           float64
horsepower     object
weight         object
model_year      int64
origin         object
dtype: object


In [58]:
print("\nMissing values:\n", df.isna().sum())


Missing values:
 mpg           0
horsepower    9
weight        0
model_year    0
origin        4
dtype: int64


In [59]:
print("\nDuplicate rows:", df.duplicated().sum())


Duplicate rows: 11


In [60]:
print("\nTarget (mpg) summary:\n", df["mpg"].describe())


Target (mpg) summary:
 count    172.000000
mean      23.494186
std        7.272505
min        9.000000
25%       18.000000
50%       23.100000
75%       28.000000
max       43.400000
Name: mpg, dtype: float64


In [61]:
print("\nOrigin counts (incl. missing):\n", df["origin"].value_counts(dropna=False))


Origin counts (incl. missing):
 origin
usa       101
japan      41
europe     26
None        4
Name: count, dtype: int64


In [62]:
print("\nHorsepower sample values:\n", df["horsepower"].head(12).tolist())


Horsepower sample values:
 [75.0, 112.0, 63.0, nan, 62.0, 68.0, nan, 68.0, 67.0, 84.0, nan, 200.0]


In [63]:
print("\nWeight sample values:\n", df["weight"].head(12).tolist())


Weight sample values:
 [2246, 2835, 2215, 2065, 1845, 1970, 2391, 2045, 3250, 2295, 3035, 4376]


## Step 4 — Clean the data

We’ll apply simple, common cleaning rules:
1. Drop exact duplicates
2. Convert numeric columns safely (`errors="coerce"` turns bad strings into NaN)
3. Standardize categorical values (strip whitespace, lowercase)
4. Handle missing values:
   - Drop missing target rows (can’t train without labels)
   - Fill missing numeric values with the median (simple baseline)
   - Fill missing categories with `"unknown"`
5. Light outlier handling:
   - Cap extreme values using an IQR-based rule (simple winsorization)

> There are more advanced approaches, but this keeps the tutorial focused and reproducible.


In [65]:
df_clean = df.copy()

# 1) Drop duplicates
df_clean = df_clean.drop_duplicates()

# 2) Coerce numerics (fix "unknown", trailing spaces, etc.)
df_clean["horsepower"] = pd.to_numeric(df_clean["horsepower"], errors="coerce")
df_clean["weight"] = pd.to_numeric(df_clean["weight"], errors="coerce")

# 3) Standardize categorical
df_clean["origin"] = df_clean["origin"].astype("string").str.strip().str.lower()
df_clean["origin"] = df_clean["origin"].fillna("unknown")

# 4) Handle missing target + fill numeric missing (this is imp)
df_clean = df_clean.dropna(subset=["mpg"])

for c in ["horsepower", "weight", "model_year"]:
    df_clean[c] = df_clean[c].fillna(df_clean[c].median())

# 5) Basic outlier capping (IQR)
def cap_iqr(s, k=1.5):
    q1, q3 = s.quantile([0.25, 0.75])
    iqr = q3 - q1
    lo, hi = q1 - k * iqr, q3 + k * iqr
    return s.clip(lo, hi)

df_clean["horsepower"] = cap_iqr(df_clean["horsepower"])
df_clean["weight"] = cap_iqr(df_clean["weight"])

print("Clean shape:", df_clean.shape)
print("\nMissing after cleaning:\n", df_clean.isna().sum())
df_clean.head()


Clean shape: (161, 5)

Missing after cleaning:
 mpg           0
horsepower    0
weight        0
model_year    0
origin        0
dtype: int64


,mpg,horsepower,weight,model_year,origin
0,26.0,75.0,2246,74,europe
1,22.0,112.0,2835,82,usa
2,34.7,63.0,2215,81,unknown
3,30.0,92.5,2065,71,unknown
4,29.8,62.0,1845,80,europe


## Step 5 — Post-clean analysis (sanity + quick relationships)

Now we re-check summaries and do simple relationships:
- Numeric summaries for scale/range
- Correlation with target (quick directional signal)
- Average `mpg` by `origin` (category-level differences)

> Correlation is not causation; it’s just a fast diagnostic.


In [66]:
print(df_clean[["mpg", "horsepower", "weight", "model_year"]].describe())

print("\nCorrelation with mpg:")
print(df_clean[["mpg", "horsepower", "weight", "model_year"]].corr(numeric_only=True)["mpg"].sort_values(ascending=False))

print("\nAverage mpg by origin:")
print(df_clean.groupby("origin")["mpg"].mean().sort_values(ascending=False))


              mpg  horsepower       weight  model_year
count  161.000000  161.000000   161.000000  161.000000
mean    23.654037   98.701863  2931.403727   76.012422
std      7.259571   29.274529   812.996482    3.650321
min      9.000000   46.000000  1773.000000   70.000000
25%     18.000000   75.000000  2254.000000   73.000000
50%     23.200000   92.500000  2774.000000   76.000000
75%     28.100000  110.000000  3420.000000   79.000000
max     43.400000  162.500000  4952.000000   82.000000

Correlation with mpg:
mpg           1.000000
model_year    0.479012
horsepower   -0.743483
weight       -0.813449
Name: mpg, dtype: float64

Average mpg by origin:
origin
japan      29.269444
europe     27.807692
unknown    25.925000
usa        20.293684
Name: mpg, dtype: float64


## Step 6 — Dummy variables with `pd.get_dummies`

Linear regression needs numeric features.
We’ll one-hot encode `origin` into 0/1 columns.

We set `drop_first=True` to avoid the “dummy variable trap”
(perfect multicollinearity when you include all categories).


In [69]:
df_model = df_clean[["mpg", "horsepower", "weight", "model_year", "origin"]].copy()

df_model = pd.get_dummies(df_model, columns=["origin"], drop_first=True)

df_model.head()


,mpg,horsepower,weight,model_year,origin_japan,origin_unknown,origin_usa
0,26.0,75.0,2246,74,False,False,False
1,22.0,112.0,2835,82,False,False,True
2,34.7,63.0,2215,81,False,True,False
3,30.0,92.5,2065,71,False,True,False
4,29.8,62.0,1845,80,False,False,False


## Step 7 — Train/test split + Feature scaling

Why scale?
- Linear regression can work without scaling, but scaling helps:
  - coefficient comparability across features
  - numerical stability in some cases
  - consistent preprocessing habit for models that require it

Important:
- Fit the scaler on **training data only**
- Transform both train and test with the same scaler


In [70]:
X = df_model.drop(columns=["mpg"])
y = df_model["mpg"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

X_train_scaled.head()


,horsepower,weight,model_year,origin_japan,origin_unknown,origin_usa
139,-0.714550,-0.409535,1.671610,-0.551677,-0.09167,0.830747
82,-1.122541,-0.978471,1.116719,1.812654,-0.09167,-1.203736
96,-0.408557,-1.031809,-1.102847,1.812654,-0.09167,-1.203736
131,-0.408557,-0.433241,1.671610,-0.551677,-0.09167,0.830747
36,0.509422,0.325340,0.839273,-0.551677,-0.09167,0.830747


## Step 8 — Train and evaluate a Multiple Linear Regression model

We’ll use:
- **MAE** (mean absolute error): average absolute error in mpg units
- **RMSE** (root mean squared error): penalizes larger errors more
- **R²**: proportion of variance explained (higher is better, can be negative on bad models)

Note on RMSE:
- In newer scikit-learn, use `root_mean_squared_error`
- Otherwise compute `np.sqrt(mean_squared_error(...))`


In [71]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)

pred = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, pred)
r2 = r2_score(y_test, pred)

if HAS_RMSE:
    rmse = root_mean_squared_error(y_test, pred)
else:
    rmse = np.sqrt(mean_squared_error(y_test, pred))

print("MAE :", round(mae, 3))
print("RMSE:", round(rmse, 3))
print("R^2 :", round(r2, 3))


MAE : 2.957
RMSE: 4.017
R^2 : 0.668


In [74]:
pred

array([21.38617893, 15.05584467, 23.78415668, 28.52091997, 27.50676497,
       22.81778618, 29.355446  , 22.8450898 , 25.3022593 , 15.40670586,
       31.17765191, 30.35774425, 29.4940196 , 29.07188006, 22.04211401,
       17.28665019, 32.22902471, 23.90387337, 27.58476753, 27.24150851,
       12.13708922, 20.99028693, 26.27935468, 29.69572198, 33.12438222,
       27.00369387, 30.81866897, 20.16377469, 26.32085342, 26.79874695,
       21.83209516, 25.32919757, 25.70359597, 20.14056159, 36.45276674,
       14.14640144, 15.07116818, 24.52569973, 26.8927127 , 21.22131472,
       13.69239013])

In [75]:
y_test

105    22.5
108    18.0
142    25.5
55     29.0
94     30.5
29     25.0
101    25.4
51     21.0
100    26.4
143    15.5
19     23.7
84     34.4
15     31.0
66     27.5
24     28.0
30     15.0
128    26.0
148    24.0
98     36.4
16     30.9
75     13.0
18     20.5
12     24.0
9      32.0
31     36.1
152    20.0
97     43.4
56     23.0
132    27.0
104    28.1
137    25.0
78     28.0
60     19.0
115    13.0
2      34.7
123    14.0
45     16.0
42     23.2
69     30.0
90     20.6
26     14.0
Name: mpg, dtype: float64

## Step 9 — Interpret coefficients (optional but useful)

Because we scaled features, coefficients are roughly comparable:
- Larger magnitude ⇒ bigger association with mpg (holding others constant)
- Positive ⇒ higher predicted mpg
- Negative ⇒ lower predicted mpg

Caution:
- With correlated predictors, coefficients can shift
- These are associations, not necessarily causal effects


In [72]:
coef = pd.Series(model.coef_, index=X_train_scaled.columns).sort_values(key=np.abs, ascending=False)
coef.to_frame("coefficient")


,coefficient
weight,-4.309380
model_year,2.550119
origin_usa,-1.261686
origin_japan,0.442634
origin_unknown,0.398780
horsepower,-0.143321


## Mini exercises 
1. Change `random_state` and see how metrics vary.
2. Remove outlier capping and compare RMSE.
3. Try different fill strategies (mean vs median).
4. Add interaction terms (e.g., weight * horsepower) and re-run.

## Wrap-up
You now have a simple, realistic regression tutorial:
- messy inputs
- cleaning
- dummy variables
- scaling
- train/test evaluation


## Optional

In [73]:
import statsmodels.api as sm

# Add intercept term for statsmodels
X_sm = sm.add_constant(X_train_scaled)

ols = sm.OLS(y_train, X_sm).fit()
print(ols.summary())


The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.811
Model:                            OLS   Adj. R-squared:                  0.801
Method:                 Least Squares   F-statistic:                     80.74
Date:                Wed, 18 Feb 2026   Prob (F-statistic):           1.53e-38
Time:                        01:19:01   Log-Likelihood:                -308.56
No. Observations:                 120   AIC:                             631.1
Df Residuals:                     113   BIC:                             650.6
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|